In [1]:
import timeit
import pandas as pd
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf

import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR

# Clean and tokenize text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords') # download stopwords corpus
nltk.download('punkt') # download punkt tokenizer

2023-05-31 12:20:42.873094: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Hannah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Hannah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
%cd /Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline

/Users/Hannah/Documents/VU/Msc/Thesis/Coding/Pipeline


## XGBOOST WORD2VEC BASED

In [8]:
df = pd.read_csv('Word2VecData.csv') # To read in the csv file that only has posts > 40 reactions
#df = pd.read_csv('transformers_dataNew.csv') # To read in the csv file with all posts
df = df[df['entropy'] < 1.3]
#df = df[df['reactions_count'] > 30]

X = df.iloc[:, -160:]
#X = df['prep_content']
y = df['entropy']

# In the first step we will split the data in training and remaining dataset

X_train, X_rem, y_train, y_rem = train_test_split(X , y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

# create a Random Forest regressor object
# Define XGBoost model
XBBmodel = XGBRegressor(random_state = 42 )
XBBmodel.fit(X_train, y_train)

# make predictions on validation set
y_val_pred = XBBmodel.predict(X_valid)

# evaluate performance on validation set
val_mse = mean_squared_error(y_valid, y_val_pred)
print("Validation MSE:", val_mse)

# Define hyperparameter search space
params = {
    'max_depth': [2, 3, 5, 7],
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(XBBmodel, param_grid=params, cv=10, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Print best hyperparameters
print('Best parameters:', grid_search.best_params_)

# Evaluate model on test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('Test MSE (tuned):', mse)

Validation MSE: 0.15502395006189973
Fitting 10 folds for each of 48 candidates, totalling 480 fits
Best parameters: {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 200}
Test MSE (tuned): 0.12524550120570935


## XGBOOST TRANSFORMER BASED

In [4]:
df = pd.read_csv('transformers_dataNew.csv')
#df = df[df['reactions_count'] > 30]
X = df.iloc[:, -768:-1]
#X = df['prep_content']
y = df['entropy']

# In the first step we will split the data in training and remaining dataset

X_train, X_rem, y_train, y_rem = train_test_split(X , y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

# create a Random Forest regressor object
# Define XGBoost model
XBBmodel = XGBRegressor(random_state = 42 )
XBBmodel.fit(X_train, y_train)

# make predictions on validation set
y_val_pred = XBBmodel.predict(X_valid)

# evaluate performance on validation set
val_mse = mean_squared_error(y_valid, y_val_pred)
print("Validation MSE:", val_mse)

# Define hyperparameter search space
params = {
    'max_depth': [2, 3, 5, 7],
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(XBBmodel, param_grid=params, cv=10, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Print best hyperparameters
print('Best parameters:', grid_search.best_params_)

# Evaluate model on test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('Test MSE (tuned):', mse)

Validation MSE: 0.1283247562518864
Fitting 10 folds for each of 48 candidates, totalling 480 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
Test MSE (tuned): 0.11597619922017657


High RC: Best parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 150}
Test MSE (tuned): 0.099076315465922

## Random Forest

In [6]:
df = pd.read_csv('transformers_dataNew.csv')
#df = df[df['reactions_count'] > 30]
X = df.iloc[:, -768:-1]
#X = df['prep_content']
y = df['entropy']

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X , y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

# define the parameter grid to search
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 3, 4, 5]
}

# create a Random Forest regressor object
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# make predictions on validation set
y_val_pred = rf_model.predict(X_valid)

# evaluate performance on validation set
val_mse = mean_squared_error(y_valid, y_val_pred)
print("Validation MSE:", val_mse)

# create a GridSearchCV object and fit it to the data
rf_grid = GridSearchCV(rf_model, param_grid, cv=10, n_jobs=-1, scoring='neg_mean_squared_error')
rf_grid.fit(X_train, y_train)

# get the best model and print its hyperparameters
best_rf = rf_grid.best_estimator_
print("Best hyperparameters:", rf_grid.best_params_)

# evaluate the model on the test set
y_pred = best_rf.predict(X_test)
test_mse = mean_squared_error(y_pred, y_test)
print("Test MSE (tuned):", test_mse)

Validation MSE: 0.12120121822668806
Best hyperparameters: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
Test MSE (tuned): 0.11569953013927566


Normal: 
Best hyperparameters: {'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 150}
Test MSE (tuned): 0.12172077754626903

HIGH RC: Test MSE (tuned): 0.10767828457147623

## SVR TRANSFORMERS BASED

In [5]:
df = pd.read_csv('transformers_dataNew.csv')
#df = df[df['reactions_count'] > 30]
X = df.iloc[:, -768:-1]
#X = df['prep_content']
y = df['entropy']

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X , y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

# initialize and train the SVM regressor
svm = LinearSVR(random_state=42)
svm.fit(X_train, y_train)

# make predictions on validation set
y_val_pred = svm.predict(X_valid)

# evaluate performance on validation set
val_mse = mean_squared_error(y_valid, y_val_pred)
print("Validation MSE:", val_mse)

# tune hyperparameters with GridSearchCV
params = {"C": [0.1, 1, 10], "epsilon": [0.1, 0.2, 0.3, 0.4]}
svm_grid = GridSearchCV(svm, params, cv=10, scoring='neg_mean_squared_error')
svm_grid.fit(X_valid, y_valid)

# print best hyperparameters and corresponding performance on validation set
print("Best hyperparameters:", svm_grid.best_params_)
best_svm = svm_grid.best_estimator_
best_svm.fit(X_train, y_train)
y_pred = best_svm.predict(X_test)
test_mse = mean_squared_error(y_pred, y_test)
print("Test MSE (tuned):", test_mse)


/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Validation MSE: 0.11521928257610053


/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn

Best hyperparameters: {'C': 0.1, 'epsilon': 0.4}
Test MSE (tuned): 0.13304399589882418


/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


NORMAL:
Validation MSE: 0.13351362680688386
Best hyperparameters: {'C': 0.1, 'epsilon': 0.3}
Test MSE (tuned): 0.13003759970726186

HIGH RC: Validation MSE: 0.3023060723692162
Best hyperparameters: {'C': 0.1, 'epsilon': 0.3}
Test MSE (tuned): 0.11590169331844272

## SVR WORD2VEC BASED

In [7]:
df = pd.read_csv('Word2VecData.csv') # To read in the csv file that only has posts > 40 reactions
#df = pd.read_csv('transformers_dataNew.csv') # To read in the csv file with all posts
df = df[df['entropy'] < 1.3]
#df = df[df['reactions_count'] > 30]
X = df.iloc[:, -160:]
#df.dropna(subset=['content'], inplace=True)
y = df['entropy']

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X , y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

# initialize and train the SVM regressor
svm = LinearSVR(random_state=42)
svm.fit(X_train, y_train)

# make predictions on validation set
y_val_pred = svm.predict(X_valid)

# evaluate performance on validation set
val_mse = mean_squared_error(y_valid, y_val_pred)
print("Validation MSE:", val_mse)

# tune hyperparameters with GridSearchCV
params = {"C": [0.1, 1, 10], "epsilon": [0.1, 0.2, 0.3, 0.4]}
svm_grid = GridSearchCV(svm, params, cv=10, scoring='neg_mean_squared_error')
svm_grid.fit(X_valid, y_valid)

# print best hyperparameters and corresponding performance on validation set
print("Best hyperparameters:", svm_grid.best_params_)
best_svm = svm_grid.best_estimator_
best_svm.fit(X_train, y_train)
y_pred = best_svm.predict(X_test)
test_mse = mean_squared_error(y_pred, y_test)
print("Test MSE (tuned):", test_mse)


/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Validation MSE: 0.13032697609761304


/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn

Best hyperparameters: {'C': 0.1, 'epsilon': 0.4}


/opt/anaconda3/envs/modelenv/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Test MSE (tuned): 0.1290451478366421
